# Reductions approach by Agarwal et al. - Recruiting data

This notebook contains the implementation of the in-processing algorithm introduced in [A Reductions Approach to Fair Classification
](http://proceedings.mlr.press/v80/agarwal18a.html) by Pleiss et al. (2018) as part of the [FairLearn tool box](https://github.com/fairlearn/).

Ihe intervention achieves a fair classification as the minimisation of the prediction error under a general form of linear constraint, which addresses Demographic Parity and Equalised Odds as special cases. The optimisation is solved by a sequence of cost-sensitive classification problems.

In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

from fairlearn.metrics import (
    equalized_odds_difference,
    demographic_parity_difference,
)
from fairlearn.reductions import ExponentiatedGradient  # noqa
from fairlearn.reductions import (
    DemographicParity,
    EqualizedOdds,
    TruePositiveRateDifference,
)
from helpers.metrics import accuracy
from helpers.plot import group_bar_plots

In [ ]:
from helpers import export_plot

## Load data
We have committed preprocessed data to the repository for reproducibility and we load it here. Check out hte preprocessing notebook for details on how this data was obtained.

In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

In [ ]:
data_dir = artifacts_dir / "data" / "recruiting"

train = pd.read_csv(data_dir / "processed" / "train.csv")
val = pd.read_csv(data_dir / "processed" / "val.csv")
test = pd.read_csv(data_dir / "processed" / "test.csv")

In [ ]:
race = train.drop("employed_yes", axis=1)["race_white"].apply(
    lambda race_white: "black" if race_white == 0 else "white"
)

## Load original model

For maximum reproducibility we can also load the baseline model from disk, but the code used to train can be found in the baseline model notebook.

In [ ]:
baseline_model = joblib.load(
    artifacts_dir / "models" / "recruiting" / "baseline.pkl"
)

Get predictions on the test data

In [ ]:
bl_test_probs = baseline_model.predict_proba(
    test.drop("employed_yes", axis=1)
)[:, 1]
bl_test_labels = (bl_test_probs > 0.5).astype(float)

## Demographic parity

We first address demographic parity. In order to do so, we learn the reductions algorithm based on the true labels of the training data. We then apply the learnt fair calssifier to predict the test data labels and analyse the outcomes for fairness and accuracy. 

In [ ]:
constraint = DemographicParity()

### Learn intervention

Since the FairLearn implementation of Agarwal et al. requires the classifier it will be learning to have a sample_weight argument, we cannot learn the type of neural net our baseline model is based on with this approach. Instead we choose a random forest classifier, and learn it on the training data. 

In [ ]:
classifier = RandomForestClassifier(n_estimators=500, max_depth=10)

Since the training procedure is lengthy we load the resulting predicted labels on the test data from a previously learnt fair model. The user is encouraged to reproduce these results however by running the commented out code for training the fair model.  

In [ ]:
test_pred_labels = np.load(
    artifacts_dir / "models" / "recruiting" / "agarwal_dp.npy"
)

For maximum reproducability we set the random seed. This makes sure we generate the same model from the training procedure.

In [ ]:
np.random.seed(42)

The underlying implementation applies the exponential gradient reduction algorithm for fair classification from Agarwal et al.

In [ ]:
# mitigator = ExponentiatedGradient(classifier, constraint)
# mitigator.fit(
#     train.drop("employed_yes", axis=1),
#     train.employed_yes.values,
#     sensitive_features=race,
# )

Generate predictions from fair classifier on test data.

In [ ]:
# test_pred_labels = mitigator.predict(test.drop("employed_yes", axis=1))

Analyse fairness and accuracy on test data.

In [ ]:
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_dpd = demographic_parity_difference(
    test_employed, bl_test_labels, sensitive_features=test_race,
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_dpd = demographic_parity_difference(
    test_employed, test_pred_labels, sensitive_features=test_race,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline demographic parity: {bl_test_dpd:.3f}")
print(f"Demographic parity: {test_dpd:.3f}\n")

Consider accuracy on the female / male subgroups

In [ ]:
white_acc = accuracy(
    test_pred_labels[test.race_white == 1],
    test.employed_yes[test.race_white == 1],
)
black_acc = accuracy(
    test_pred_labels[test.race_white == 0],
    test.employed_yes[test.race_white == 0],
)

mean_black_score = test_pred_labels[test.race_white == 0].mean()
mean_white_score = test_pred_labels[test.race_white == 1].mean()

print(f"Black accuracy: {black_acc:.3f}")
print(f"White accuracy: {white_acc:.3f}")
print(f"Mean black score: {mean_black_score:.3f}")
print(f"Mean white score: {mean_white_score:.3f}")

In [ ]:
dp_bar = group_bar_plots(
    test_pred_labels,
    test.race_white.map({0: "Black", 1: "White"}),
    title="Predictions by race",
    xlabel="Proportion predicted successful",
)
dp_bar

In [ ]:
export_plot(dp_bar, "agarwal-dp.json")

## Equalised odds

We'll now repeat the process for equalised odds, which requires us changing the constraint and leads to learning a fair classifier addressing equalised odds. There are no further modifcations to the existing parameter choices required. 

In [ ]:
constraint = EqualizedOdds()

Again, we load the the resulting test label predictions from a previously learnt model. The code that generated that model is commented out below.

In [ ]:
test_pred_labels = np.load(
    artifacts_dir / "models" / "recruiting" / "agarwal_eo.npy"
)

Learn intervention

In [ ]:
# np.random.seed(42)
# mitigator = ExponentiatedGradient(classifier, constraint)
# mitigator.fit(
#     train.drop("employed_yes", axis=1),
#     train.employed_yes.values,
#     sensitive_features=race,
# )

Generate predictions from fair classifier test data.

In [ ]:
# test_pred_labels = mitigator.predict(test.drop("employed_yes", axis=1))

Analyse fairness and accuracy on test data

In [ ]:
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_eod = equalized_odds_difference(
    test_employed, bl_test_labels, sensitive_features=test_race,
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_eod = equalized_odds_difference(
    test_employed, test_pred_labels, sensitive_features=test_race,
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equalised odds (dist.): {bl_test_eod:.3f}")
print(f"Equalised odds (dist.): {test_eod:.3f}\n")

In [ ]:
eo_bar = group_bar_plots(
    test_pred_labels,
    test.race_white.map({0: "Black", 1: "White"}),
    groups=test.employed_yes,
    group_names=["Not employed", "Employed"],
    title="Predictions by race and outcome",
    xlabel="Proportion predicted successful",
    ylabel="Outcome",
)
eo_bar

In [ ]:
export_plot(eo_bar, "agarwal-eo.json")

## Equal opportunity
We'll now repeat the process for equalised opportunity, which only requires us changing the constraint.

In [ ]:
constraint = TruePositiveRateDifference()

Again, we load the the resulting test label predictions from a previously learnt model. The code that generated that model is commented out below.

In [ ]:
test_pred_labels = np.load(
    artifacts_dir / "models" / "recruiting" / "agarwal_eopp.npy"
)

Learn intervention

In [ ]:
# np.random.seed(42)
# mitigator = ExponentiatedGradient(classifier, constraint)
# mitigator.fit(
#     train.drop("employed_yes", axis=1),
#     train.employed_yes.values,
#     sensitive_features=race,
# )

Generate predictions from fair classifier test data.

In [ ]:
# test_pred_labels = mitigator.predict(test.drop("employed_yes", axis=1))

Analyse fairness and accuracy on test data

In [ ]:
test_race = test.race_white.values
test_employed = test.employed_yes.values
mask = test_race == 1

# baseline metrics
bl_test_acc = accuracy(test_employed, bl_test_probs)
bl_test_eoppd = equalized_odds_difference(
    test_employed[test.employed_yes == 1],
    bl_test_labels[test.employed_yes == 1],
    sensitive_features=test_race[test.employed_yes == 1],
)

# new model metrics
test_acc = accuracy(test_employed, test_pred_labels)
test_eoppd = equalized_odds_difference(
    test_employed[test.employed_yes == 1],
    test_pred_labels[test.employed_yes == 1],
    sensitive_features=test_race[test.employed_yes == 1],
)

print(f"Baseline accuracy: {bl_test_acc:.3f}")
print(f"Accuracy: {test_acc:.3f}\n")

print(f"Baseline equal opportunity: {bl_test_eoppd:.3f}")
print(f"Equal opportunity: {test_eoppd:.3f}\n")

In [ ]:
eopp_bar = group_bar_plots(
    test_pred_labels[test.employed_yes == 1],
    test.race_white[test.employed_yes == 1].map({0: "Black", 1: "White"}),
    title="Predictions by race for successful applicants",
    xlabel="Proportion predicted successful",
)
eopp_bar

In [ ]:
export_plot(eopp_bar, "agarwal-eopp.json")